Without comments

Just for point. because we ain't got enough samples

### Load

In [1]:
import pandas as pd, numpy as np

In [2]:
df = pd.read_csv('data/train.tsv', sep='\t', quotechar=' ', header = None)
df.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply', 'label', 'confidence']
df_test = pd.read_csv('data/public.tsv', sep='\t', quotechar = ' ', header = None)
df_test.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply']

In [3]:
df.fillna('', inplace=True)
df_test.fillna('', inplace=True)

### Preprocessing

#### Labeling target

In [4]:
def label_enc(x ,reverse = False):
    if reverse == False:
        if x == 'bad':
            return 0
        elif x == 'neutral':
            return 1
        else:
            return 2
    else:
        if x == 0:
            return 'bad'
        elif x == 1:
            return 'neutral'
        else:
            return 'good'

In [5]:
df['label'] = df['label'].apply(label_enc)

In [6]:
list_sentences_train = df[['context_2', 'context_1', 'context_0', 'reply']].as_matrix()
list_sentences_test = df_test[['context_2', 'context_1', 'context_0', 'reply']].as_matrix()
y = df['label'].values

In [7]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.callbacks import EarlyStopping, ModelCheckpoint

/home/denis/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
# define network parameters
max_features = 64
maxlen = 100

In [22]:
tokenizer = keras_text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True,
                                 split=' ', char_level=True, oov_token=None)

tokenizer.fit_on_texts(list(list_sentences_train))
# train data
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
X_t = keras_seq.pad_sequences(list_tokenized_train, maxlen=maxlen)
# test data
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = keras_seq.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [3]:


def build_model(conv_layers = 2, 
                dilation_rates = [0, 2, 4, 8, 16], 
                embed_size = 256):
    
    inp = Input(shape=(None, ))
    x = Embedding(input_dim = len(tokenizer.word_counts)+1, 
                  output_dim = embed_size)(inp)
    prefilt_x = Dropout(0.25)(x)
    out_conv = []
    # dilation rate lets us use ngrams and skip grams to process 
    for dilation_rate in dilation_rates:
        x = prefilt_x
        for i in range(2):
            if dilation_rate>0:
                x = Conv1D(16*2**(i), 
                           kernel_size = 3, 
                           dilation_rate = dilation_rate,
                          activation = 'relu',
                          name = 'ngram_{}_cnn_{}'.format(dilation_rate, i)
                          )(x)
            else:
                x = Conv1D(16*2**(i), 
                           kernel_size = 1,
                          activation = 'relu',
                          name = 'word_fcl_{}'.format(i))(x)
        out_conv += [Dropout(0.5)(GlobalMaxPool1D()(x))]
    x = concatenate(out_conv, axis = -1)    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='kullback_leibler_divergence',
                  optimizer='adam',
                  metrics=['accuracy', mcor,recall, f1])
    return model

In [11]:
from sklearn.model_selection import train_test_split
print('Distribution of Total Positive Labels (important for validation)')
X_t_train, X_t_test, y_train, y_test = train_test_split(X_t, y, 
                                                        test_size = 0.25, 
                                                        random_state = 2018)
print('Training:', X_t_train.shape)
print('Testing:', X_t_test.shape)

Distribution of Total Positive Labels (important for validation)
Training: (73149, 100)
Testing: (24384, 100)


In [12]:
model = build_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    25359872    input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, None, 256)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
word_fcl_0 (Conv1D)             (None, None, 16)     4112        dropout_1[0][0]                  
__________________________________________________________________________________________________
ngram_2_cn

In [13]:
batch_size = 64 # large enough that some other labels come in
epochs = 100

file_path="best_weights.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

callbacks_list = [checkpoint, early] #early
model.fit(X_t_train, y_train, 
          validation_data=(X_t_test, y_test),
          batch_size=batch_size, 
          epochs=epochs, 
          shuffle = True,
          callbacks=callbacks_list)

Train on 73149 samples, validate on 24384 samples
Epoch 1/100
23616/73149 [========>.....................] - ETA: 33s - loss: 0.0206 - acc: 0.1156

KeyboardInterrupt: 